In [1]:
from pyspark.sql.functions import hour,minute, second, col, avg, when
from pyspark.sql import SparkSession

In [2]:
spark= SparkSession.builder.appName("Learn").getOrCreate()
cpuDetails=spark.read.csv('usrINFO.csv',header=True, inferSchema=True)
cpuDetails.show()

+--------------------+-------------------+-------------------+-------------------+-------------------+
|           user_name|          idle_time|      working_hours|         start_time|           end_time|
+--------------------+-------------------+-------------------+-------------------+-------------------+
|  sahil24c@gmail.com|2019-10-24 05:05:00|2019-10-24 05:50:00|2019-10-24 08:30:02|2019-10-24 19:25:02|
|magadum.iranna@gm...|2019-10-24 02:15:00|2019-10-24 08:39:59|2019-10-24 08:30:02|2019-10-24 19:25:01|
|  yathink3@gmail.com|2019-10-24 01:30:00|2019-10-24 09:24:59|2019-10-24 08:30:02|2019-10-24 19:25:01|
|  shelkeva@gmail.com|2019-10-24 00:30:00|2019-10-24 09:10:01|2019-10-24 08:45:01|2019-10-24 18:25:02|
|puruissimple@gmai...|2019-10-24 03:15:00|2019-10-24 07:19:59|2019-10-24 08:50:02|2019-10-24 19:25:01|
|sangita.awaghad19...|2019-10-24 01:55:00|2019-10-24 08:40:00|2019-10-24 08:50:01|2019-10-24 19:25:01|
|vaishusawant143@g...|2019-10-24 00:35:00|2019-10-24 09:55:00|2019-10-24 

In [3]:
dataIdleHour=cpuDetails.withColumn('hours2Sec',hour(cpuDetails['idle_time'])*3600)
dataIdleMin=cpuDetails.withColumn('Minute2Sec',minute(cpuDetails['idle_time'])*60)
dataIdleSec=cpuDetails.withColumn('Seconds',second(cpuDetails['idle_time']))
hours=dataIdleHour.select('user_name','hours2Sec')
Minutes=dataIdleMin.select('user_name','Minute2Sec')
seconds=dataIdleSec.select('user_name','Seconds')
hours.show(5)
Minutes.show(5)
seconds.show(5)

+--------------------+---------+
|           user_name|hours2Sec|
+--------------------+---------+
|  sahil24c@gmail.com|    18000|
|magadum.iranna@gm...|     7200|
|  yathink3@gmail.com|     3600|
|  shelkeva@gmail.com|        0|
|puruissimple@gmai...|    10800|
+--------------------+---------+
only showing top 5 rows

+--------------------+----------+
|           user_name|Minute2Sec|
+--------------------+----------+
|  sahil24c@gmail.com|       300|
|magadum.iranna@gm...|       900|
|  yathink3@gmail.com|      1800|
|  shelkeva@gmail.com|      1800|
|puruissimple@gmai...|       900|
+--------------------+----------+
only showing top 5 rows

+--------------------+-------+
|           user_name|Seconds|
+--------------------+-------+
|  sahil24c@gmail.com|      0|
|magadum.iranna@gm...|      0|
|  yathink3@gmail.com|      0|
|  shelkeva@gmail.com|      0|
|puruissimple@gmai...|      0|
+--------------------+-------+
only showing top 5 rows



In [4]:
hourAmin=hours.join(Minutes, on=['user_name'], how='inner')
hourAmin.show(5)

+--------------------+---------+----------+
|           user_name|hours2Sec|Minute2Sec|
+--------------------+---------+----------+
|  sahil24c@gmail.com|    18000|       300|
|magadum.iranna@gm...|     7200|       900|
|  yathink3@gmail.com|     3600|      1800|
|  shelkeva@gmail.com|        0|      1800|
|puruissimple@gmai...|    10800|       900|
+--------------------+---------+----------+
only showing top 5 rows



In [5]:
hourAminAsec=hourAmin.join(seconds,on=['user_name'],how='inner')
hourAminAsec.show(5)

+--------------------+---------+----------+-------+
|           user_name|hours2Sec|Minute2Sec|Seconds|
+--------------------+---------+----------+-------+
|  sahil24c@gmail.com|    18000|       300|      0|
|magadum.iranna@gm...|     7200|       900|      0|
|  yathink3@gmail.com|     3600|      1800|      0|
|  shelkeva@gmail.com|        0|      1800|      0|
|puruissimple@gmai...|    10800|       900|      0|
+--------------------+---------+----------+-------+
only showing top 5 rows



In [6]:
val=hourAminAsec.withColumn('Avgvalue', (hourAminAsec['hours2Sec']+hourAminAsec['Minute2Sec']+hourAminAsec['Seconds'])/3600)
val.show(5)

+--------------------+---------+----------+-------+-----------------+
|           user_name|hours2Sec|Minute2Sec|Seconds|         Avgvalue|
+--------------------+---------+----------+-------+-----------------+
|  sahil24c@gmail.com|    18000|       300|      0|5.083333333333333|
|magadum.iranna@gm...|     7200|       900|      0|             2.25|
|  yathink3@gmail.com|     3600|      1800|      0|              1.5|
|  shelkeva@gmail.com|        0|      1800|      0|              0.5|
|puruissimple@gmai...|    10800|       900|      0|             3.25|
+--------------------+---------+----------+-------+-----------------+
only showing top 5 rows



In [7]:
AvgIdle=val.select(avg('Avgvalue'))
AvgIdle.show()

+-------------+
|avg(Avgvalue)|
+-------------+
|      2.03125|
+-------------+



In [9]:
val.filter(val['Avgvalue']>2.0312).select('user_name','Avgvalue').show()

+--------------------+------------------+
|           user_name|          Avgvalue|
+--------------------+------------------+
|  sahil24c@gmail.com| 5.083333333333333|
|magadum.iranna@gm...|              2.25|
|puruissimple@gmai...|              3.25|
|     you@example.com|2.5833333333333335|
|vishnu23kumar@gma...|              4.25|
|narsimharaj.kasu0...|2.0833333333333335|
|sargampandey27oct...|              4.75|
|ayush.saraf47@gma...| 4.333333333333333|
|mr.kundare@gmail.com|10.083333333333334|
|surajpj7852@gmail...|2.1666666666666665|
|ruchikachile30199...|3.5833333333333335|
|farooqbassam4@gma...|               2.5|
|“shivnajalisangal...| 9.833333333333334|
| addyp1911@gmail.com|3.1666666666666665|
|dipakalagate1991@...|3.1666666666666665|
|gaikwadr576@gmail...|3.0833333333333335|
|tekina.makin@gmai...| 9.416666666666666|
|mishrasushil889@g...|               3.5|
| blsonalib@gmail.com|2.5833333333333335|
|hakepratiksha55@g...| 9.416666666666666|
+--------------------+------------